In [384]:
import pandas as pd
import gzip
import json
import re
from collections import Counter
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import stanza
from tqdm import tqdm
from nltk.collocations import *

### Выкачиваем данные

In [113]:
reviews = []
for line in open ('Office_Products_5.json', 'r', encoding='utf-8'):
    reviews.append(json.loads(line))

In [114]:
metadata = []
for line in open ('meta_Office_Products.json', 'r', encoding='utf-8'):
    metadata.append(json.loads(line))

In [115]:
metadata = {elem['asin'] : {'category' : elem['category'], 
                            'title' : elem['title'], 
                            'brand': elem['brand'] if 'title' in elem.keys() else ''} for elem in metadata}

In [116]:
data = []
for review in reviews:
    try:
        d = {}
        d['review'] = review['reviewText']
        d['asin'] = review['asin']
        d['category'] = metadata[review['asin']]['category']
        d['title'] = metadata[review['asin']]['title']
        d['brand'] = metadata[review['asin']]['brand']
        data.append(d)
    except:
        continue

In [170]:
df = pd.DataFrame(data)
df
df.to_csv('data_office_products.csv')

# 3 способа найти упоминания товаров в отзывах

#### 1. Использовать модели для извлечения именованных сущностей (например, stanza)  
  
Минусы:
- обработка занимает много времени 
- вероятно, получится не совсем идеально
- названия офисной техники специфические, не факт, что пользователи будут в отзывах писать именно названия, а не тип продукта (например, в отзывах часто встречается чисто название без упоминания типа продукта)  
  
Плюсы: 
- скорее всего получится вытащить упоминание названий продукта из отзывов
- извлеченные названия можно использовать для дальнейшего объединения с типами продукта, что позволит посмотреть извлечь из отзыва больше полезной информации о товаре

#### 2. Извлечь типы товаров из названий или категорий и затем посмотреть на n-граммы с ними в отзывах  
  
Минусы: 
- вероятно, не получится отследить названия товаров, поскольку они часто встречаются без упоминания типа продукта (например: "I bought my first HP12C in about 1984 or so")
- пользователи могут называть товар иначе (синонимом)  
  
Плюсы: 
- типы продуктов достаточно легко извлечь из их названий

#### 3. Векторизовать корпус c помощью word2vec  
  
Минусы: 
- упоминания товаров могут состоять из >1 слова, следовательно надо думать над n-граммами
- этот способ не кажется сильно хорошим для выбранного корпуса, поскольку названия типов товаров зачастую не имеют синонимов => вряд ли получится сгруппировать упоминания одного товара и его синонимичных названий  
  
Плюсы: 
- Можно сгруппировать упоминания одного товара и его синонимичных названий
- Достаточно быстро 

### Что выбираем?  
Попробуем объединить два способа: 
1) достанем названия продуктов с помощью stanza  
2) достанем типы продуктов из названий и/или категорий

Затем посмотрим в отзывах на биграммы с вытащенными сущностями. Плюс использование этих двух способов в совокупности может помочь объединить название товара с его типом, которые в какой-то мере можно считать синонимичными. 

Выделим категории товаров

In [27]:
categories = []
for item in df['category']:
    categories = categories + [category for category in item]
categories = set(categories)

Достанем типы товаров из названий категорий

In [230]:
goods = []
for category in categories:
    doc = nlp(category)
    for sent in doc.sentences:
        for word in sent.words:
            if word.upos == 'NOUN':
                goods.append(word.lemma.lower())

#### Попробуем также выделить названия типов товаров из их названий.  
Названия товаров в этой категории достаточно длинные и имеют большое количество доп. информации о товаре. Тип товара обычно стоит: 
- либо перед первой запятой, после которой идут уточнения
- либо перед "for" или "with"
- либо в конце названия (если нет запятых)
- либо в конце перед артикулом, который в скобках 

In [161]:
lemmatizer = WordNetLemmatizer()

#### Извлечем типы товаров из названий
    

In [269]:
goods_titles = []
for title in df['title']:
    title = title.split(',')[0] #делим по запятой и берем первую часть, так как в ней содержится название
    title = re.sub(r' .+&quot', r' ', title)
    title = re.sub(r'[ \(]+[0-9A-Z\-\# ]+[ \)]+', r' ', title) #убираем все цифры и коды
    title = re.sub(r' [a-zA-Z0-9]+-[a-z]+ ', ' ', title)
    title = re.sub(r'\(.+\)', '', title)
    title = re.sub(r' +', ' ', title)
    title = title[:-1] if title[-1] == ' ' else title
    try:
        match = re.search(r' ([A-Z][a-z]+)( [for|with|pro])', title)
        if match:
            product = lemmatizer.lemmatize(match[1].lower())
            goods_titles.append(product)
        elif re.search(r'[0-9]', title.split(' ')[-1]) or title.split(' ')[-1].startswith('-'):
            if re.fullmatch('[a-z]+', lemmatizer.lemmatize(title.split(' ')[-2].lower())):
                product = lemmatizer.lemmatize(title.split(' ')[-2].lower())
                goods_titles.append(product)
            else:
                product = lemmatizer.lemmatize(title.split(' ')[-3].lower())
                goods_titles.append(product)
        elif re.fullmatch('[a-z]+', lemmatizer.lemmatize(title.split(' ')[-1].lower())):
            product = lemmatizer.lemmatize(title.split(' ')[-1].lower())
            goods_titles.append(product)
        else:
            goods_titles.append('')
    except:
        goods_titles.append('')
        continue
df['product'] = goods_titles

Получилось, если честно, не ососбо чисто

In [243]:
set(goods_titles)

{'',
 'accessory',
 'adapter',
 'adf',
 'adhesive',
 'aerosol',
 'all',
 'arm',
 'badge',
 'bag',
 'ball',
 'ballpoint',
 'base',
 'bell',
 'binder',
 'black',
 'blade',
 'block',
 'board',
 'body',
 'book',
 'bookend',
 'box',
 'bridge',
 'builder',
 'bundle',
 'calculator',
 'calendar',
 'callisto',
 'canister',
 'card',
 'cardstock',
 'cart',
 'cartridge',
 'case',
 'cement',
 'center',
 'chair',
 'chairmat',
 'chalk',
 'charger',
 'clear',
 'clip',
 'cm',
 'color',
 'companion',
 'compatible',
 'construction',
 'converter',
 'copier',
 'copyholder',
 'cosmic',
 'count',
 'cover',
 'cube',
 'cup',
 'cushion',
 'cutter',
 'cyan',
 'darkness',
 'design',
 'desk',
 'dictionary',
 'disappear',
 'dispenser',
 'divider',
 'document',
 'doorstop',
 'dpi',
 'drawer',
 'easel',
 'edge',
 'electric',
 'envelope',
 'eprinter',
 'epson',
 'eraser',
 'fastener',
 'fax',
 'featuring',
 'fellowes',
 'file',
 'filer',
 'film',
 'fine',
 'finger',
 'finish',
 'flag',
 'fluid',
 'folder',
 'footrest'

Но если посмотреть на самые частотные, становится лучше

In [244]:
dict_titles = Counter(goods_titles)
dict_titles.most_common(50)

[('printer', 2944),
 ('label', 1429),
 ('binder', 901),
 ('dispenser', 889),
 ('folder', 887),
 ('pad', 828),
 ('marker', 823),
 ('tape', 822),
 ('note', 807),
 ('pen', 777),
 ('stapler', 750),
 ('paper', 673),
 ('cartridge', 654),
 ('box', 619),
 ('sharpener', 556),
 ('envelope', 511),
 ('board', 475),
 ('scanner', 450),
 ('rest', 425),
 ('pencil', 351),
 ('phone', 342),
 ('laminator', 293),
 ('kit', 291),
 ('set', 251),
 ('shredder', 246),
 ('pocket', 244),
 ('card', 238),
 ('calculator', 237),
 ('eraser', 235),
 ('system', 234),
 ('sheet', 230),
 ('divider', 222),
 ('pack', 208),
 ('tab', 207),
 ('pouch', 204),
 ('notebook', 193),
 ('organizer', 192),
 ('protector', 189),
 ('holder', 188),
 ('tank', 175),
 ('notetabs', 175),
 ('maker', 173),
 ('pockettabs', 145),
 ('ticket', 144),
 ('scale', 137),
 ('flag', 129),
 ('machine', 128),
 ('edge', 123),
 ('liquid', 121),
 ('black', 117)]

#### Сравним с товарами, извлеченными из категорий

In [277]:
dict_categories = Counter(goods)
dict_categories.most_common(50)

[('&amp', 41),
 ('printer', 17),
 ('card', 16),
 ('tape', 15),
 ('supply', 14),
 ('label', 13),
 ('envelope', 11),
 ('holder', 11),
 ('accessory', 11),
 ('paper', 10),
 ('stand', 10),
 ('desk', 10),
 ('pad', 10),
 ('office', 10),
 ('index', 10),
 ('folder', 10),
 ('pencil', 9),
 ('file', 9),
 ('binder', 8),
 ('adhesive', 8),
 ('scanner', 8),
 ('product', 8),
 ('eraser', 7),
 ('correction', 7),
 ('box', 7),
 ('laser', 6),
 ('pen', 6),
 ('stamp', 6),
 ('organizer', 6),
 ('stapler', 6),
 ('dispenser', 6),
 ('business', 6),
 ('planner', 6),
 ('refill', 5),
 ('cart', 5),
 ('inkjet', 5),
 ('storage', 5),
 ('notebook', 5),
 ('mailer', 5),
 ('glue', 5),
 ('binding', 4),
 ('book', 4),
 ('stick', 4),
 ('pocket', 4),
 ('calendar', 4),
 ('board', 4),
 ('badge', 4),
 ('form', 4),
 ('chair', 4),
 ('photo', 4)]

In [276]:
print('Тип товара не удалось извлечь у ', len(set(df[df['product']=='']['title'])), 'товаров. \nКоличество отзывов на данные товары в базе: ', len(df[df['product']=='']))

Тип товара не удалось извлечь у  47 товаров. 
Количество отзывов на данные товары в базе:  993


### Теперь воспользуемся Stanza для извлечения названий товаров из отзывов.

In [29]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,lemma,pos,ner')

2022-12-15 01:00:18 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2022-12-15 01:01:12 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| ner       | ontonotes |

2022-12-15 01:01:12 INFO: Use device: cpu
2022-12-15 01:01:12 INFO: Loading: tokenize
2022-12-15 01:01:12 INFO: Loading: pos
2022-12-15 01:01:12 INFO: Loading: lemma
2022-12-15 01:01:12 INFO: Loading: ner
2022-12-15 01:01:13 INFO: Done loading processors!


Так как Stanza работает достаточно долго, не будем обрабатывать всю базу, а возьмем из нее 5 разных (согласно извлеченным типам из названий): printer, calculator, stapler, marker, binder, dispenser

In [285]:
new_df = df[df['product'].isin(['printer', 'binder', 'dispenser', 'folder', 'stapler', 'calculator'])]

In [286]:
product_name = []
for text in tqdm(new_df['review']): 
    products = []
    doc = nlp(text)
    for sent in doc.sentences:
        for ent in sent.ents:
            if ent.type == 'PRODUCT':
                products.append(ent.text)
    product_name.append(products)

100%|██████████| 7379/7379 [8:22:28<00:00,  4.09s/it]     


In [295]:
new_df['NE'] = [list(set(names)) for names in product_name] 

<ipython-input-295-29185988adeb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['NE'] = [list(set(names)) for names in product_name]


In [297]:
new_df.to_csv('new_data_office.csv')

### Собираем n-граммы  

Сойдемся на том, что в отзыве автор пишет об одном товаре, то есть, если в названии пользователь упоминает название продукта и оно есть в официальном названии продукта, то мы имеем право заменить упомянутое название на тип продукта.

In [308]:
def review_preprocessing (review, product, title, NE): 
    for el in set(NE):
        if el in title:
            review = review.replace(el, ' '+product+' ')
        else:
            continue
            
    new_review = ''
    tokens = word_tokenize(review.lower())
    for i in tokens:
        if i.isalpha():
            new_review = new_review + lemmatizer.lemmatize(i) + ' '
    return new_review

In [309]:
changed_reviews = []
for review, product, title, NE in zip(new_df['review'],  new_df['product'], new_df['title'], new_df['NE']):
    new_review = review_preprocessing (review, product, title, NE)
    changed_reviews.append(new_review)

In [313]:
new_df['new_review'] = changed_reviews

<ipython-input-313-bcb0ecf26fc8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['new_review'] = changed_reviews


#### Биграммы

In [314]:
product_names = ['printer', 'binder', 'dispenser', 'folder', 'stapler', 'calculator']

In [438]:
bigrams = ngrams(' '.join(new_df['new_review']).split(), 2)

In [439]:
bi_dict = Counter(bigrams)

In [440]:
product_bigrams = {'printer':[], 'binder':[], 'dispenser':[], 'folder':[], 'stapler':[], 'calculator':[]}
    
for bigram in bi_dict.most_common():
    for product in product_bigrams.keys():
        if product in bigram[0]:
            product_bigrams[product].append(bigram)

In [441]:
for product in product_names:
    print(product) 
    for bigram in product_bigrams[product][:10]:
        print(bigram)
    print('\n')

printer
(('the', 'printer'), 6406)
(('this', 'printer'), 3885)
(('printer', 'is'), 1898)
(('printer', 'and'), 1337)
(('printer', 'i'), 1335)
(('printer', 'for'), 937)
(('a', 'printer'), 871)
(('laser', 'printer'), 707)
(('printer', 'to'), 692)
(('printer', 'that'), 684)


binder
(('the', 'binder'), 571)
(('this', 'binder'), 473)
(('binder', 'is'), 307)
(('a', 'binder'), 223)
(('binder', 'i'), 183)
(('binder', 'for'), 140)
(('binder', 'and'), 128)
(('binder', 'that'), 116)
(('binder', 'it'), 112)
(('binder', 'with'), 110)


dispenser
(('the', 'dispenser'), 534)
(('tape', 'dispenser'), 491)
(('dispenser', 'is'), 288)
(('this', 'dispenser'), 213)
(('dispenser', 'i'), 99)
(('dispenser', 'and'), 93)
(('dispenser', 'it'), 84)
(('dispenser', 'that'), 81)
(('note', 'dispenser'), 66)
(('dispenser', 'to'), 59)


folder
(('the', 'folder'), 718)
(('file', 'folder'), 571)
(('these', 'folder'), 449)
(('folder', 'are'), 413)
(('hanging', 'folder'), 221)
(('folder', 'i'), 197)
(('folder', 'and'), 197)

Многие частотные биграммы не особо информативны, поскольку содержат в себе предлоги, артикли и тд.  
Создадим список стоп-слов, чтобы отсеить по нему неинформативные биграммы

In [367]:
stopwords = ['the', 'this', 'is', 'and', 'i', 'for', 'a', 'to',
                'that', 'it', 'on', 'of', 'at', 'from', 'with',
                'can', 'are', 'in', 'will', 'would','these', 'if', 
                'but', 'so', 'or', 'because', 'up', 'ha']

Отфильтруем биграммы

In [378]:
product_bigrams_2 = {'printer':[], 'binder':[], 'dispenser':[], 'folder':[], 'stapler':[], 'calculator':[]}
    
for bigram in bi_dict.most_common():
    for product in product_bigrams_2.keys():
        if (product in bigram[0] and bigram[0][0] not in stopwords 
            and bigram[0][1] not in stopwords and bigram[0][0] != bigram[0][1]):
            product_bigrams_2[product].append(bigram)

Уже гораздо лучше

In [437]:
for product in product_names:
    print(product) 
    for bigram in product_bigrams_2[product][:10]:
        print(bigram)
    print('\n')

printer
(('laser', 'printer'), 707)
(('epson', 'printer'), 582)
(('photo', 'printer'), 460)
(('printer', 'wa'), 427)
(('inkjet', 'printer'), 370)
(('other', 'printer'), 339)
(('hp', 'printer'), 296)
(('canon', 'printer'), 294)
(('printer', 'doe'), 283)
(('my', 'printer'), 276)


binder
(('view', 'binder'), 74)
(('ring', 'binder'), 71)
(('other', 'binder'), 71)
(('duty', 'binder'), 54)
(('jones', 'binder'), 50)
(('quality', 'binder'), 38)
(('inch', 'binder'), 31)
(('avery', 'binder'), 29)
(('my', 'binder'), 29)
(('basic', 'binder'), 27)


dispenser
(('tape', 'dispenser'), 491)
(('note', 'dispenser'), 66)
(('handband', 'dispenser'), 38)
(('dispenser', 'tape'), 35)
(('little', 'dispenser'), 29)
(('dispenser', 'work'), 26)
(('dispenser', 'wa'), 22)
(('dispenser', 'doe'), 21)
(('dispenser', 'which'), 19)
(('dispenser', 'you'), 18)


folder
(('file', 'folder'), 571)
(('hanging', 'folder'), 221)
(('manila', 'folder'), 129)
(('folder', 'they'), 85)
(('folder', 'have'), 68)
(('folder', 'you'), 

In [380]:
df_bigrams = pd.DataFrame({el : [' '.join(i[0]) for i in product_bigrams_2[el][:200]] for el in product_names})

In [381]:
df_bigrams

,printer,binder,dispenser,folder,stapler,calculator
0,laser printer,view binder,tape dispenser,file folder,my stapler,graphing calculator
1,epson printer,ring binder,note dispenser,hanging folder,swingline stapler,scientific calculator
2,photo printer,other binder,handband dispenser,manila folder,stapler doe,hp calculator
3,printer wa,duty binder,dispenser tape,folder they,standard stapler,financial calculator
4,inkjet printer,jones binder,little dispenser,folder have,stapler wa,basic calculator
...,...,...,...,...,...,...
195,printer durabrite,serviceable binder,dispenser refillable,cheap folder,typical stapler,he calculator
196,printer being,strong binder,dispenser along,different folder,usual stapler,calculator apparently
197,printer requires,binder lie,dispenser designed,folder together,attractive stapler,calculator mandatory
198,printer again,mitt binder,dispenser release,partition folder,easy stapler,versatile calculator


#### Триграммы

In [360]:
trigrams = ngrams(' '.join(new_df['new_review']).split(), 3)

In [363]:
tri_dict = Counter(trigrams)

In [364]:
product_trigrams = {'printer':[], 'binder':[], 'dispenser':[], 'folder':[], 'stapler':[], 'calculator':[]}

for trigram in tri_dict.most_common():
    for product in product_trigrams.keys():
        if (product in trigram[0] and trigram[0][0] != trigram[0][1] and trigram[0][1] != trigram[0][2]):
            product_trigrams[product].append(trigram)

Не очень хочется убирать стоп-слова, так как некоторые триграммы с ними достаточно неплохие

In [372]:
df_trigrams = pd.DataFrame({el : [' '.join(i[0]) for i in product_trigrams[el][:200]] for el in product_names})

In [373]:
df_trigrams

,printer,binder,dispenser,folder,stapler,calculator
0,this printer is,of the binder,of the dispenser,of the folder,this stapler is,this calculator is
1,the printer is,this binder is,the dispenser is,these folder are,of the stapler,the calculator is
2,of the printer,the binder is,this tape dispenser,the folder are,the stapler is,a graphing calculator
3,the printer and,wilson jones binder,dispenser is a,hanging file folder,with this stapler,of the calculator
4,with this printer,heavy duty binder,this dispenser is,these file folder,stapler is a,the hp calculator
...,...,...,...,...,...,...
195,this printer you,the binder can,dispenser on your,in one folder,stapler work well,with all calculator
196,printer for home,binder is not,dispenser make it,folder that will,a swingline stapler,this calculator big
197,printer it doe,the binder cover,tape dispenser will,standard hanging folder,stapler with a,calculator big number
198,laser printer that,binder i do,tape dispenser on,use hanging folder,a stapler go,calculator for you


### Ранжирование n-грамм с помощью коллокационных метрик

In [374]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

In [414]:
finder = BigramCollocationFinder.from_words(' '.join(new_df['new_review']).split())

### PMI

In [415]:
pmi_bigrams = {'printer':[], 'binder':[], 'dispenser':[], 'folder':[], 'stapler':[], 'calculator':[]}
for bigram in finder.score_ngrams(bigram_measures.pmi):
    for product in product_names:
        if (product in bigram[0] and bigram[0][0] != bigram[0][1] 
            and bigram[0][0] not in stopwords and bigram[0][1] not in stopwords):
                pmi_bigrams[product].append(bigram)

In [416]:
for product in product_names:
    print(product) 
    for bigram in pmi_bigrams[product][0:50]:
        print(bigram)
    print('\n')

printer
(('ahp', 'printer'), 6.043137262229832)
(('anaemic', 'printer'), 6.043137262229832)
(('ausb', 'printer'), 6.043137262229832)
(('beastmode', 'printer'), 6.043137262229832)
(('boxthe', 'printer'), 6.043137262229832)
(('btter', 'printer'), 6.043137262229832)
(('capacityblack', 'printer'), 6.043137262229832)
(('cartridgethis', 'printer'), 6.043137262229832)
(('clunkyno', 'printer'), 6.043137262229832)
(('cmyblack', 'printer'), 6.043137262229832)
(('comparablebrother', 'printer'), 6.043137262229832)
(('comparisonthe', 'printer'), 6.043137262229832)
(('considerationsthis', 'printer'), 6.043137262229832)
(('crated', 'printer'), 6.043137262229832)
(('cxi', 'printer'), 6.043137262229832)
(('daisywheel', 'printer'), 6.043137262229832)
(('departmental', 'printer'), 6.043137262229832)
(('devicesthe', 'printer'), 6.043137262229832)
(('doggiest', 'printer'), 6.043137262229832)
(('duplexerthe', 'printer'), 6.043137262229832)
(('exhale', 'printer'), 6.043137262229832)
(('expson', 'printer'), 6

### raw_freq

In [417]:
rf_bigrams = {'printer':[], 'binder':[], 'dispenser':[], 'folder':[], 'stapler':[], 'calculator':[]}
for bigram in finder.score_ngrams(bigram_measures.raw_freq):
    for product in product_names:
        if (product in bigram[0] and bigram[0][0] != bigram[0][1] 
            and bigram[0][0] not in stopwords and bigram[0][1] not in stopwords):
                rf_bigrams[product].append(bigram)

In [418]:
for product in product_names:
    print(product) 
    for bigram in rf_bigrams[product][0:50]:
        print(bigram)
    print('\n')

printer
(('laser', 'printer'), 0.0004750944981100378)
(('epson', 'printer'), 0.0003910961780764385)
(('photo', 'printer'), 0.00030911381772364554)
(('printer', 'wa'), 0.00028693826123477533)
(('inkjet', 'printer'), 0.000248635027299454)
(('other', 'printer'), 0.00022780344393112138)
(('hp', 'printer'), 0.0001989080218395632)
(('canon', 'printer'), 0.00019756404871902562)
(('printer', 'doe'), 0.00019017219655606887)
(('my', 'printer'), 0.0001854682906341873)
(('wireless', 'printer'), 0.00018345233095338093)
(('new', 'printer'), 0.00014447711045779084)
(('great', 'printer'), 0.00014313313733725326)
(('printer', 'you'), 0.00013641327173456532)
(('printer', 'which'), 0.00012834943301133977)
(('your', 'printer'), 0.00012767744645107097)
(('workforce', 'printer'), 9.878202435951281e-05)
(('printer', 'itself'), 9.407811843763125e-05)
(('color', 'printer'), 9.071818563628727e-05)
(('good', 'printer'), 8.601427971440571e-05)
(('old', 'printer'), 8.198236035279295e-05)
(('printer', 'have'), 7.86

### likelihood_ratio

In [419]:
lr_bigrams = {'printer':[], 'binder':[], 'dispenser':[], 'folder':[], 'stapler':[], 'calculator':[]}
for bigram in finder.score_ngrams(bigram_measures.likelihood_ratio):
    for product in product_names:
        if (product in bigram[0] and bigram[0][0] != bigram[0][1] 
            and bigram[0][0] not in stopwords and bigram[0][1] not in stopwords):
                lr_bigrams[product].append(bigram)

In [420]:
for product in product_names:
    print(product) 
    for bigram in lr_bigrams[product][0:50]:
        print(bigram)
    print('\n')

printer
(('laser', 'printer'), 4337.908207532014)
(('inkjet', 'printer'), 1977.5440037485907)
(('epson', 'printer'), 1720.6376901940112)
(('photo', 'printer'), 1038.2792985470176)
(('canon', 'printer'), 868.6867049642465)
(('hp', 'printer'), 819.4020082643071)
(('other', 'printer'), 735.275073641096)
(('new', 'printer'), 635.879060723498)
(('wireless', 'printer'), 564.2953722599759)
(('printer', 'itself'), 551.8026973642714)
(('workforce', 'printer'), 514.713613843623)
(('printer', 'doe'), 480.52692014655787)
(('you', 'printer'), 479.86537633420585)
(('printer', 'wa'), 398.19639179383046)
(('multifunction', 'printer'), 371.27909255629)
(('jet', 'printer'), 311.65955884535015)
(('brother', 'printer'), 300.8277979053021)
(('great', 'printer'), 300.5128568776336)
(('format', 'printer'), 277.82529172016837)
(('old', 'printer'), 274.05840934239177)
(('have', 'printer'), 263.40303134900654)
(('pixma', 'printer'), 261.8320926411314)
(('not', 'printer'), 259.72807114281574)
(('monochrome', 'pr

Мне показались более качественными последние биграммы

#### Теперь посмотрим на триграммы

In [421]:
finder_3 = TrigramCollocationFinder.from_words(' '.join(new_df['new_review']).split())

### PMI

In [430]:
pmi_trigrams = {'printer':[], 'binder':[], 'dispenser':[], 'folder':[], 'stapler':[], 'calculator':[]}

for trigram in finder_3.score_ngrams(trigram_measures.pmi):
    for product in product_names:
        if (product in trigram[0] and trigram[0][0] != trigram[0][1] and trigram[0][1] != trigram[0][2]):
            pmi_trigrams[product].append(trigram)

In [431]:
for product in product_names:
    print(product) 
    for trigram in pmi_trigrams[product][0:50]:
        print(trigram)
    print('\n')

printer
(('comparisonthe', 'printer', 'mfx'), 26.548201547049615)
(('diablo', 'daisywheel', 'printer'), 26.548201547049615)
(('emptyplus', 'morethe', 'printer'), 26.548201547049615)
(('feederwireless', 'printingtouchscreen', 'printer'), 26.548201547049615)
(('printer', 'eddy', 'kleinnewton'), 26.548201547049615)
(('upusb', 'portcolor', 'printer'), 26.548201547049615)
(('alexmark', 'pinnacle', 'printer'), 25.548201547049615)
(('sampler', 'printer', 'controlas'), 25.548201547049615)
(('thissamsung', 'printer', 'fwis'), 25.548201547049615)
(('upgradeable', 'printer', 'firmwarethe'), 25.548201547049615)
(('printer', 'roger', 'meyerlife'), 24.96323904632846)
(('inket', 'printer', 'jean'), 24.548201547049615)
(('printer', 'kindly', 'remembered'), 24.548201547049615)
(('printer', 'chug', 'gallon'), 24.226273452162253)
(('pcl', 'fontsthis', 'printer'), 23.96323904632846)
(('printer', 'screeched', 'horribly'), 23.96323904632846)
(('snazzier', 'printer', 'featuring'), 23.96323904632846)
(('thiss

### raw_freq

In [432]:
rf_trigrams = {'printer':[], 'binder':[], 'dispenser':[], 'folder':[], 'stapler':[], 'calculator':[]}

for trigram in finder_3.score_ngrams(trigram_measures.pmi):
    for product in product_names:
        if (product in trigram[0] and trigram[0][0] != trigram[0][1] and trigram[0][1] != trigram[0][2]):
            rf_trigrams[product].append(trigram)

In [433]:
for product in product_names:
    print(product) 
    for trigram in rf_trigrams[product][0:50]:
        print(trigram)
    print('\n')

printer
(('comparisonthe', 'printer', 'mfx'), 26.548201547049615)
(('diablo', 'daisywheel', 'printer'), 26.548201547049615)
(('emptyplus', 'morethe', 'printer'), 26.548201547049615)
(('feederwireless', 'printingtouchscreen', 'printer'), 26.548201547049615)
(('printer', 'eddy', 'kleinnewton'), 26.548201547049615)
(('upusb', 'portcolor', 'printer'), 26.548201547049615)
(('alexmark', 'pinnacle', 'printer'), 25.548201547049615)
(('sampler', 'printer', 'controlas'), 25.548201547049615)
(('thissamsung', 'printer', 'fwis'), 25.548201547049615)
(('upgradeable', 'printer', 'firmwarethe'), 25.548201547049615)
(('printer', 'roger', 'meyerlife'), 24.96323904632846)
(('inket', 'printer', 'jean'), 24.548201547049615)
(('printer', 'kindly', 'remembered'), 24.548201547049615)
(('printer', 'chug', 'gallon'), 24.226273452162253)
(('pcl', 'fontsthis', 'printer'), 23.96323904632846)
(('printer', 'screeched', 'horribly'), 23.96323904632846)
(('snazzier', 'printer', 'featuring'), 23.96323904632846)
(('thiss

### likelihood_ratio

In [427]:
lr_trigrams = {'printer':[], 'binder':[], 'dispenser':[], 'folder':[], 'stapler':[], 'calculator':[]}

for trigram in finder_3.score_ngrams(trigram_measures.likelihood_ratio):
    for product in product_names:
        if (product in trigram[0] and trigram[0][0] != trigram[0][1] and trigram[0][1] != trigram[0][2]):
            lr_trigrams[product].append(trigram)

In [429]:
for product in product_names:
    print(product) 
    for trigram in lr_trigrams[product][0:50]:
        print(trigram)
    print('\n')

printer
(('printer', 'if', 'you'), 24299.351551983418)
(('of', 'the', 'printer'), 23365.06701390706)
(('on', 'the', 'printer'), 23111.311427881377)
(('the', 'printer', 'of'), 19440.504218693706)
(('in', 'the', 'printer'), 19008.486693660565)
(('this', 'printer', 'is'), 18489.028854459677)
(('i', 'the', 'printer'), 17676.339986297968)
(('the', 'printer', 'is'), 17348.18532206544)
(('with', 'the', 'printer'), 17238.227041105678)
(('with', 'this', 'printer'), 16713.136398989598)
(('this', 'printer', 'a'), 16250.915896174745)
(('a', 'the', 'printer'), 15836.9119371161)
(('the', 'this', 'printer'), 15765.720420314672)
(('i', 'this', 'printer'), 15683.060721120986)
(('recommend', 'this', 'printer'), 15661.682405165733)
(('a', 'this', 'printer'), 15641.65251967662)
(('the', 'printer', 'the'), 15637.795350594499)
(('i', 'have', 'printer'), 15577.318982224402)
(('from', 'the', 'printer'), 15556.714169621282)
(('of', 'this', 'printer'), 15236.17325879713)
(('printer', 'i', 'have'), 15221.8075302

Триграммы мне не особо понравились, поэтому выведем только биграммы по каждому товару

In [435]:
for product in product_names:
    print(product)
    print('---')
    for bi in lr_bigrams[product][:50]:   
        print(' '.join(bi[0]))
    print('\n')


printer
---
laser printer
inkjet printer
epson printer
photo printer
canon printer
hp printer
other printer
new printer
wireless printer
printer itself
workforce printer
printer doe
you printer
printer wa
multifunction printer
jet printer
brother printer
great printer
format printer
old printer
have printer
pixma printer
not printer
monochrome printer
wa printer
printer driver
printer copier
previous printer
print printer
paper printer
be printer
printer which
printer us
your printer
printer be
do printer
printer come
printer scanner
home printer
samsung printer
they printer
kodak printer
out printer
when printer
printing printer
printer folder
doe printer
just printer
printer cable
very printer


binder
---
view binder
jones binder
ring binder
duty binder
avery binder
other binder
zipper binder
inch binder
basic binder
durable binder
particular binder
binder clip
sturdy binder
binder ring
white binder
binder unusable
quality binder
you binder
average binder
binder itself
ordinary bind